
## <center> DeepSeek R1 Distill高效微调入门实战

### 一、unsloth快速使用入门

#### 1.借助unsloth进行模型推理

In [66]:
! pwd
! pip install unsloth
from unsloth import FastLanguageModel

/data/jupyterfile/wei


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.internal-mirrors.ucloud.cn/simple


- 尝试用unsloth进行LLama模型推理

&emsp;&emsp;首先设置关键参数，并读取模型：

In [67]:
max_seq_length = 2048 
dtype = None 
# load_in_4bit = False
load_in_4bit = True

> 注，若显存不足，则可以load_in_4bit = True，运行4 bit量化版。

In [68]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./model/deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.151 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.91s/it]


./model/deepseek-ai/DeepSeek-R1-Distill-Llama-8B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


> 在INT4量化情况下，8B模型推理仅需7G左右显存。

此时model就是读取进来的DeepSeek R1 8B蒸馏模型：

In [69]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

而tokenizer则是分词器：

In [70]:
tokenizer

LlamaTokenizerFast(name_or_path='./model/deepseek-ai/DeepSeek-R1-Distill-Llama-8B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	128000: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=F

将模型调整为推理模式：

In [71]:
FastLanguageModel.for_inference(model) 

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

然后即可和模型进行对话：

In [72]:
question = "请问如何证明根号2是无理数？"

然后这里我们首先需要借助分词器，将输入的问题转化为标记索引：

In [73]:
inputs = tokenizer([question], return_tensors="pt").to("cuda")

In [74]:
inputs

{'input_ids': tensor([[128000,  15225,  57107, 109425, 125544, 102831,  18476,     17,  21043,
          43292,  22649,   9039,  11571]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

最后再带入inputs进行对话

In [75]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

此时得到的回复也是词索引：

In [76]:
outputs

tensor([[128000,  15225,  57107,  ...,     13,    220,  78388]],
       device='cuda:0')

同样需要分词器将其转化为文本：

In [77]:
response = tokenizer.batch_decode(outputs)

In [78]:
response

['<｜begin▁of▁sentence｜>请问如何证明根号2是无理数？或者有没有其他方法？\n\n嗯，我现在要证明根号2是无理数，对吧？或者有没有其他方法？首先，我记得证明√2是无理数的方法是通过假设它是有理数，然后推导出矛盾。让我先回忆一下这个过程。\n\n好，首先假设√2是一个有理数，也就是可以表示成两个整数的比值，也就是√2 = a/b，其中a和b都是正整数，并且b不等于0，而且这个分数已经是最简形式，也就是说a和b没有除了1以外的公约数。\n\n接下来，我可以把这个等式两边平方，得到2 = a² / b²，也就是a² = 2b²。这样的话，a²必须是2的倍数，因为右边是2b²，而b²是整数，所以a²也是整数，进而a也是整数。\n\n既然a²是2的倍数，那么a也必须是2的倍数，对吗？因为如果一个数的平方是偶数，那么这个数本身也必须是偶数。所以，a可以表示为2k，其中k是一个整数。\n\n现在，我把a代入原来的等式√2 = a/b，也就是√2 = (2k)/b。那么，两边平方得到2 = (4k²)/b²，进而得到b² = 2k²。\n\n同样地，这意味着b²也是2的倍数，所以b也必须是2的倍数。也就是说，b可以表示为2m，其中m是一个整数。\n\n现在，我已经有了a = 2k和b = 2m，代入√2 = a/b，也就是√2 = (2k)/(2m) = k/m。这样的话，√2 = k/m，其中k和m都是整数，而且k和m也是互质的，因为原来的a和b已经是最简形式。\n\n现在，这就导致了一个问题，因为如果√2是一个有理数，也就是可以表示成两个整数的比值，那么它必然是最简形式的，也就是k和m互质。但是，根据上面的推导，√2 = k/m，而k和m都必须是偶数吗？\n\n不对，等一下，k和m是否是偶数呢？因为a = 2k，所以k可以是任何整数，同样，b = 2m，所以m也是整数。现在，假设k和m都是偶数，那么我们可以继续分解下去，得到k = 2n，m = 2p，这样的话，√2 = (2n)/(2p) = n/p，这样就会导致√2 = n/p，其中n和p都是整数，这样就可以无限继续下去，这意味着√2可以无限分解下去，这显然是不对的，因为任何数都不能无限分解下去，除非它为0，而√2显然不等于0。\n\n因此，这说明我们的假设有问题，也就是说，如果√2是一个有理数，那么它就

In [79]:
print(response[0])

<｜begin▁of▁sentence｜>请问如何证明根号2是无理数？或者有没有其他方法？

嗯，我现在要证明根号2是无理数，对吧？或者有没有其他方法？首先，我记得证明√2是无理数的方法是通过假设它是有理数，然后推导出矛盾。让我先回忆一下这个过程。

好，首先假设√2是一个有理数，也就是可以表示成两个整数的比值，也就是√2 = a/b，其中a和b都是正整数，并且b不等于0，而且这个分数已经是最简形式，也就是说a和b没有除了1以外的公约数。

接下来，我可以把这个等式两边平方，得到2 = a² / b²，也就是a² = 2b²。这样的话，a²必须是2的倍数，因为右边是2b²，而b²是整数，所以a²也是整数，进而a也是整数。

既然a²是2的倍数，那么a也必须是2的倍数，对吗？因为如果一个数的平方是偶数，那么这个数本身也必须是偶数。所以，a可以表示为2k，其中k是一个整数。

现在，我把a代入原来的等式√2 = a/b，也就是√2 = (2k)/b。那么，两边平方得到2 = (4k²)/b²，进而得到b² = 2k²。

同样地，这意味着b²也是2的倍数，所以b也必须是2的倍数。也就是说，b可以表示为2m，其中m是一个整数。

现在，我已经有了a = 2k和b = 2m，代入√2 = a/b，也就是√2 = (2k)/(2m) = k/m。这样的话，√2 = k/m，其中k和m都是整数，而且k和m也是互质的，因为原来的a和b已经是最简形式。

现在，这就导致了一个问题，因为如果√2是一个有理数，也就是可以表示成两个整数的比值，那么它必然是最简形式的，也就是k和m互质。但是，根据上面的推导，√2 = k/m，而k和m都必须是偶数吗？

不对，等一下，k和m是否是偶数呢？因为a = 2k，所以k可以是任何整数，同样，b = 2m，所以m也是整数。现在，假设k和m都是偶数，那么我们可以继续分解下去，得到k = 2n，m = 2p，这样的话，√2 = (2n)/(2p) = n/p，这样就会导致√2 = n/p，其中n和p都是整数，这样就可以无限继续下去，这意味着√2可以无限分解下去，这显然是不对的，因为任何数都不能无限分解下去，除非它为0，而√2显然不等于0。

因此，这说明我们的假设有问题，也就是说，如果√2是一个有理数，那么它就可以被表示成一个无限可分的比值，这显然是不可

至此我们就完成了unsloth模型推理流程。

- 尝试使用unsloth调用Qwen模型

&emsp;&emsp;类似的，我们也可以使用unsloth调用Qwen蒸馏模型

In [80]:
model_qwen, tokenizer_qwen = FastLanguageModel.from_pretrained(
    model_name = "./model/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.151 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.89s/it]


./model/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B does not have a padding token! Will use pad_token = <|vision_pad|>.


In [81]:
FastLanguageModel.for_inference(model_qwen) 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((

In [82]:
inputs = tokenizer_qwen([question], return_tensors="pt").to("cuda")

outputs = model_qwen.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer_qwen.batch_decode(outputs)

print(response[0])

<｜begin▁of▁sentence｜>请问如何证明根号2是无理数？我知道有利用代数和几何的方法，但不知道具体怎么证明。

嗯，证明根号2是无理数，这应该是数学中一个经典的证明例子吧。我记得以前老师上课的时候讲过，可能用了反证法。让我先回忆一下反证法的基本步骤：假设结论相反，然后推出矛盾，从而证明原命题成立。

首先，根号2是一个无理数，也就是说，它不能表示成两个整数相除的形式。或者说，根号2的小数展开是无限不循环的。现在，我需要用反证法来证明这一点。

那我们先假设相反的情况，也就是根号2是有理数。那么，根据有理数的定义，存在两个整数a和b，使得根号2 = a/b，其中a和b互质，也就是说它们没有公因数，除了1之外。

接下来，我可以把等式两边平方一下，得到2 = a² / b²，然后两边同时乘以b²，得到2b² = a²。这一步应该是对的吧，两边乘以b²的话，左边是2乘以b²，右边是a²。

接下来，我们可以得出a²是2的倍数，也就是说，a²是偶数。那么，a本身也是偶数，因为如果一个数的平方是偶数，那么这个数本身也必须是偶数。这个结论对吗？让我想想，比如，3是奇数，平方是9，也是奇数；2是偶数，平方是4，也是偶数。所以，如果a²是偶数，a一定也是偶数。对的，没错。

既然a是偶数，那我们可以表示a为2k，其中k是某个整数。把a=2k代入原来的等式2b² = a²，得到2b² = (2k)² = 4k²。所以，2b² = 4k²，两边同时除以2，得到b² = 2k²。

现在，我们发现b²也是偶数，那么b也必须是偶数，因为只有偶数的平方才是偶数。所以，b也是偶数。但是，这与我们最初的假设相矛盾，因为我们假设a和b是互质的，也就是它们之间没有公因数，但这里a和b都是偶数，说明它们都有一个公因数2，这和互质的假设相违背。

所以，这个矛盾说明我们的假设是错误的，也就是说，根号2不可能是有理数，所以它一定是无理数。

不过，我是不是哪里漏了什么？让我再检查一下每一步是否正确。首先，假设根号2是有理数，写成a/b，互质，对吗？是的，这没问题。然后平方得到2b² = a²，所以a²是偶数，进而a也是偶数，对吗？对的。然后设a=2k，代入得2b² =4k²，化简得b²=2k²，所以b²是偶数，b也是偶数。这样a和b都是偶数，说明它们有公因数2，和互质的假设矛盾。

嗯，这样

> 注，以下实验均以DeepSeek R1 LLama3蒸馏模型为例进行演示和讲解，若想替换为DeepSeek R1 Qwen模型，则可以直接替换模型名称即可。

#### 2.带入问答模板进行回答

- 结构化输入方法

In [83]:
prompt_style_chat = """请写出一个恰当的回答来完成当前对话任务。

### Instruction:
你是一名助人为乐的助手。

### Question:
{}

### Response:
<think>{}"""

In [84]:
question = "你好，好久不见！"

In [85]:
[prompt_style_chat.format(question, "")]

['请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>']

In [86]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [87]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

In [88]:
response = tokenizer.batch_decode(outputs)

In [89]:
response

['<｜begin▁of▁sentence｜>请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>\n嗯，用户说“你好，好久不见！”，这可能是想表达一下想和我聊天，或者只是随便打个招呼。我的回应应该要友好又亲切，显示出乐于助人的态度。同时，要保持简洁，不让对话变得冗长。\n\n首先，回应开头可以用“你好呀！”这样比较随意又亲切。然后，接下来可以问用户最近怎么样，或者有什么想聊的，鼓励用户继续说下去。这样既回应了问候，又引导对话的发展。\n\n再想想，用户可能只是想和我打个招呼，或者想找些话题聊聊，所以回应要开放一些，给用户空间继续发言。比如，可以说最近有什么有趣的事情，或者有什么想聊的话题，这样用户可以根据自己的意愿回应。\n\n总的来说，我的回应应该包括以下几点：友好的问候、询问对方的情况、邀请对方分享更多内容，这样既符合助人为乐的态度，又能有效地引导对话继续下去。\n</think>\n\n你好呀！很高兴见到你。最近怎么样？有什么想聊的吗？<｜end▁of▁sentence｜>']

In [90]:
print(response[0].split("### Response:")[1])


<think>
嗯，用户说“你好，好久不见！”，这可能是想表达一下想和我聊天，或者只是随便打个招呼。我的回应应该要友好又亲切，显示出乐于助人的态度。同时，要保持简洁，不让对话变得冗长。

首先，回应开头可以用“你好呀！”这样比较随意又亲切。然后，接下来可以问用户最近怎么样，或者有什么想聊的，鼓励用户继续说下去。这样既回应了问候，又引导对话的发展。

再想想，用户可能只是想和我打个招呼，或者想找些话题聊聊，所以回应要开放一些，给用户空间继续发言。比如，可以说最近有什么有趣的事情，或者有什么想聊的话题，这样用户可以根据自己的意愿回应。

总的来说，我的回应应该包括以下几点：友好的问候、询问对方的情况、邀请对方分享更多内容，这样既符合助人为乐的态度，又能有效地引导对话继续下去。
</think>

你好呀！很高兴见到你。最近怎么样？有什么想聊的吗？<｜end▁of▁sentence｜>


- 复杂问题测试

In [91]:
question = "请证明根号2是无理数。"

In [92]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [93]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

In [94]:
response = tokenizer.batch_decode(outputs)

In [95]:
print(response[0].split("### Response:")[1])


<think>
嗯，今天老师布置了一个证明√2是无理数的题目，我觉得挺有意思的，但也有点难。首先，我得先回忆一下什么是无理数。无理数就是不能表示成两个整数之比的数，也就是说，如果√2可以写成a/b，其中a和b都是整数，且b不等于0，那它就是有理数，否则就是无理数。

那我先假设√2是有理数，也就是存在整数a和b，使得√2 = a/b，其中b不等于0。接下来，我可以把这个等式两边同时平方，得到2 = a²/b²。这意味着a² = 2b²。也就是说，a²是2的倍数，那么a也必须是2的倍数，因为如果一个数的平方是2的倍数，那这个数本身也必须是2的倍数。所以，我可以把a写成2k，其中k是一个整数。这样，a² = (2k)² = 4k²，代入上面的等式，得到4k² = 2b²，简化一下就是2k² = b²。同样地，这里b²是2的倍数，所以b也必须是2的倍数，同样可以表示为b = 2m，其中m是一个整数。这样，b² = (2m)² = 4m²，代入等式，得到2k² = 4m²，简化后是k² = 2m²。这样，又出现了k²是2的倍数，同样k也必须是2的倍数，所以k = 2n，n是一个整数。这样，k² = (2n)² = 4n²，代入等式，得到4n² = 2m²，简化后是2n² = m²。又一次，m²是2的倍数，所以m也必须是2的倍数，m = 2p，p是一个整数。这样，m² = (2p)² = 4p²，代入等式，得到2n² = 4p²，简化后是n² = 2p²。看起来这里出现了一个循环，每次都要求某个数是2的倍数，这意味着a和b都必须是无限多个2的因子，这是不可能的，因为a和b都是有限的整数。因此，这种假设导致了矛盾，所以√2不能表示成两个整数之比，因此√2是无理数。

不过，我觉得可能哪里有问题，或者是不是还有其他方法来证明。比如，使用数学归纳法或者其他方法，但我觉得上面的方法已经足够了，因为它展示了如果√2是有理数，那么它必须是无限多个2的倍数，这是不可能的，所以结论是√2是无理数。
</think>

√2是无理数。证明如下：

假设√2是有理数，则存在整数a和b（b≠0），使得√2 = a/b。两边平方得到2 = a²/b²，即a² = 2b²。因此，a必须是2的倍数，设a = 2k，k为整数。代入得(2k)² = 2b²，即4k² = 2b²，简化为2k² = b²。同

#### 3.原始模型的医疗问题问答

- 重新设置问答模板

In [96]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>{}"""

翻译如下：

```python
prompt_style = """以下是一个任务说明，配有提供更多背景信息的输入。
请写出一个恰当的回答来完成该任务。
在回答之前，请仔细思考问题，并按步骤进行推理，确保回答逻辑清晰且准确。

### Instruction:
您是一位具有高级临床推理、诊断和治疗规划知识的医学专家。
请回答以下医学问题。
```

&emsp;&emsp;接下来我们抽取部分medical-o1-reasoning-SFT数据集中问题进行提问，并查看初始状态下模型回答结果。

In [97]:
question_1 = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"

翻译：一位61岁的女性，有长期在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但夜间没有漏尿。她接受了妇科检查和Q-tip测试。根据这些检查结果，膀胱测量（cystometry）最可能会显示她的残余尿量和逼尿肌收缩情况如何？









In [98]:
question_2 = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"

翻译：面对一位突发胸痛并放射至颈部和左臂的患者，其既往病史包括高胆固醇血症和冠状动脉疾病，同时伴有升高的肌钙蛋白I水平和心动过速，根据这些临床表现，最可能受累的冠状动脉是哪一条？

- 问答测试

In [99]:
inputs1 = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")


outputs1 = model.generate(
    input_ids=inputs1.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response1 = tokenizer.batch_decode(outputs1)

In [100]:
print(response1[0].split("### Response:")[1])


<think>
Okay, so I have this medical case to analyze. Let me break it down step by step. The patient is a 61-year-old woman with a history of involuntary urine loss when she coughs or sneezes, but she doesn't leak at night. She's had a gynecological exam and a Q-tip test. I need to figure out what cystometry would show regarding her residual volume and detrusor contractions.

First, the involuntary urine loss during activities like coughing or sneezing makes me think of stress urinary incontinence. That's usually due to urethral sphincter deficiency, especially since she's a woman. The fact that she doesn't leak at night suggests that her bladder capacity is good because she can hold it until she gets to a toilet, which is typical in stress incontinence.

Next, the gynecological exam and Q-tip test. I'm a bit rusty on the Q-tip test. I think it's used to assess urethral function. The Q-tip is a small catheter or probe that's inserted into the urethra to measure resistance. If the resi

翻译如下：

\<think>  
好的，我正在尝试分析这个医学问题。我们一步步来分解。患者是一位61岁的女性，有在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但她夜间没有漏尿。她正在接受妇科检查和Q-tip测试。问题是关于膀胱测量（cystometry）会显示她的残余尿量和逼尿肌收缩情况。

首先，我知道在像打喷嚏或咳嗽等活动中发生不自主尿液流失通常与压力性尿失禁有关。压力性尿失禁通常发生在尿道肌肉不足以在压力增大的情况下（比如咳嗽时）防止膀胱漏尿时。

接下来是Q-tip测试。根据我记得的，Q-tip是一种用于测量尿道压力曲线的尿道导管。它通常用于评估尿道功能。Q-tip测试阳性结果，即在Valsalva操作过程中尿道压力低于膀胱内压，与内源性括约肌缺陷相关，这是一种压力性尿失禁类型。

由于患者有在活动中出现不自主漏尿的病史，但夜间没有漏尿，更可能是压力性尿失禁，而不是像急迫性尿失禁那样的情况，急迫性尿失禁通常伴有夜间漏尿。因此，如果Q-tip测试阳性，提示内源性括约肌缺陷。

现在，谈到膀胱测量。膀胱测量是一种测试，旨在测量膀胱在充盈过程中的反应以及逼尿肌的收缩情况。它可以显示是否存在膀胱过度活动症（OAB），即引起急迫感和频尿的情况，或是否存在逼尿肌低活动性，导致尿潴留。

在这种情况下，患者的主要问题是压力性尿失禁，这更与无法在压力增大时保持尿液有关。膀胱测量会查看逼尿肌的收缩情况。如果逼尿肌低活动性，它将不能强有力地收缩以排空膀胱，导致残余尿量。但如果逼尿肌过度活跃，可能会收缩过度，导致急迫感。

鉴于患者有压力性尿失禁的病史和Q-tip测试阳性，提示内源性括约肌缺陷，我认为膀胱测量会显示逼尿肌的收缩是正常的。问题不在于逼尿肌收缩的能力，而是无法密封尿道以保持压力。因此，残余尿量可能是正常的，除非有明显的尿潴留，但关键发现是逼尿肌的收缩是正常的，而不是过度活跃。

等等，但会不会有残余尿量？如果患者排尿后膀胱中残留一些尿液，那就是残余尿量。但如果没有尿潴留的症状，比如膀胱饱胀或排尿困难，那么这种情况的可能性较小。主要问题是在活动中发生的尿失禁，因此逼尿肌收缩是正常的，残余尿量在正常范围内，除非有其他情况。

所以，综合来看，膀胱测量可能会显示逼尿肌的收缩正常，残余尿量正常。问题更多是在括约肌方面，而不是逼尿肌。  
\</think>

根据对患者病史和Q-tip测试结果的分析，膀胱测量最可能显示逼尿肌的收缩正常，残余尿量正常。主要问题似乎是由于内源性括约肌缺陷引起的压力性尿失禁，如Q-tip测试阳性所示。这种情况通常影响尿道括约肌在压力增大时防止漏尿的能力，而不是逼尿肌的收缩能力。因此，逼尿肌的收缩并未过度活跃，残余尿量在正常范围内。

标准答案：

在这种压力性尿失禁的情况下，膀胱测压检查（cystometry）最可能显示**正常的排尿后残余尿量**，因为压力性尿失禁通常不会影响膀胱排空功能。此外，由于压力性尿失禁主要与**身体用力**有关，而不是膀胱过度活动症（OAB），因此在测试过程中**不太可能观察到逼尿肌的非自主收缩**。

In [101]:
inputs2 = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")


outputs2 = model.generate(
    input_ids=inputs2.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response2 = tokenizer.batch_decode(outputs2)

In [102]:
print(response2[0].split("### Response:")[1])


<think>
Okay, so I have a patient presenting with sudden-onset chest pain that's radiating to the neck and left arm. Hmm, that makes me think of a cardiac issue. The past medical history includes hypercholesterolemia and coronary artery disease, which are both risk factors for heart problems. Also, elevated troponin I levels are a big indicator of myocardial injury, probably from a heart attack. Tachycardia could be a response to that injury or maybe a separate issue.

I should consider which coronary artery is most likely involved. The pain radiating to the left arm makes me think of the left anterior descending (LAD) artery, but that's just one possibility. The right coronary artery (RCA) could also be involved, especially since the pain might radiate down the right arm as well. The left circumflex artery is another option, but the typical presentation might point more towards LAD or RCA.

Troponin I elevation is more specific to the left ventricle, so if the LAD is involved, that m

翻译如下：

\<think>  
好的，我有一位患者，突然出现胸痛，并放射到颈部和左臂。这让我想到了心脏病发作，因为这些症状很经典——心绞痛或心肌梗死。左臂痛、颈部痛，有时还会伴随下颌或背部的疼痛，这些都可能与冠状动脉问题相关。

从病史来看，患者有**高胆固醇血症**（即高胆固醇）和**冠状动脉疾病**，这两个因素都是**动脉粥样硬化**的风险因素，可能导致冠状动脉发生堵塞。**肌钙蛋白I升高**是一个很大的线索，因为肌钙蛋白是心肌受损时释放的心脏酶，通常表明发生了心肌梗死。另外，患者还出现了**心动过速**，即心跳比平常快。在心肌梗死时，心脏可能会加速跳动，以试图通过增加心脏输出量来补偿被阻塞的冠状动脉。

考虑到冠状动脉，**左主冠状动脉**（LMCA）为整个左侧心脏提供血液，包括左心室，而左心室是一个关键的泵血肌肉。如果这里发生堵塞，可能导致更严重的心肌梗死，因为左心室至关重要。**右冠状动脉**为右心室和左心室下壁提供血液，这里的堵塞也是可能的，但**左主冠状动脉**通常与上述症状更相关，尤其是当肌钙蛋白升高时。

所以，将所有因素综合考虑，最可能受累的冠状动脉是**左主冠状动脉**（LMCA）。患者的病史、肌钙蛋白升高以及典型的胸痛放射症状都指向了这一动脉作为罪魁祸首。

\</think>

最可能受累的冠状动脉是**左主冠状动脉（LMCA）**。

**解释：**
- **症状：** 患者突发胸痛并放射至颈部和左臂，以及肌钙蛋白升高，提示急性冠状动脉综合症，可能是心肌梗死。
- **病史：** 高胆固醇血症和冠状动脉疾病病史是动脉粥样硬化的风险因素，可能导致冠状动脉堵塞。
- **心动过速：** 心率增加可能是心脏为补偿心肌血流减少而产生的反应。
- **冠状动脉考虑：** 左主冠状动脉供应左心室，这个肌肉对心脏功能至关重要。与右冠状动脉相比，左主冠状动脉的堵塞会导致更严重且危及生命的心肌梗死，右冠状动脉通常供应的是不那么关键的区域。

因此，症状、肌钙蛋白升高以及患者的病史强烈指向**左主冠状动脉**（LMCA）作为最可能的罪魁祸首。<｜end▁of▁sentence｜>

标准答案：

根据患者表现出的突然胸痛并放射至颈部和左臂，结合其有高胆固醇血症和冠状动脉疾病的病史，肌钙蛋白升高和心动过速，临床症状强烈提示左前降支（LAD）动脉受累。该动脉通常是引发此类症状的罪魁祸首，因为它供应了心脏的大部分区域。放射性疼痛和肌钙蛋白升高的组合表明心肌受损，这使得LAD成为最可能的致病动脉。然而，在没有进一步的诊断检查（如心电图）的情况下，最终的确诊仍需等待确认。

能够看出，在原始状态下，模型能够进行推理并给出回复，但实际上第一个回答过程并不符合医学规范，而第二个问题则直接回答错误。由此可见，在初始状态下，模型对于medical-o1-reasoning-SFT数据集问答效果并不好。

&emsp;&emsp;接下来尝试进行微调，并测试微调后模型问答效果。

### 二、最小可行性实验

&emsp;&emsp;接下来我们尝试进行模型微调，对于当前数据集而言，我们可以带入原始数据集的部分数据进行微调，也可以带入全部数据并遍历多次进行微调。对于大多数的微调实验，我们都可以从最小可行性实验入手进行微调，也就是先尝试带入少量数据进行微调，并观测微调效果。若微调可以顺利执行，并能够获得微调效果，再考虑带入更多的数据进行更大规模微调。

#### 1.数据集准备

&emsp;&emsp;这里我们直接从huggingface上下载medical-o1-reasoning-SFT数据集。

- 设置代理环境

&emsp;&emsp;由于huggingface网络受限，下载数据集前需要先进行网络环境设置。若是AutoDL服务器，则可以按照如下方式开启学术加速，从而顺利连接huggingface并进行数据集下载：

In [103]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

- 下载数据集

&emsp;&emsp;接下来使用datasets进行数据集下载

In [104]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.internal-mirrors.ucloud.cn/simple


In [105]:
import os
from datasets import load_dataset

再次确认提示词模板：

In [106]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

然后提取并设置文本生成结束的标记：

In [107]:
EOS_TOKEN = tokenizer.eos_token
tokenizer.eos_token

'<｜end▁of▁sentence｜>'

然后定义函数，用于对medical-o1-reasoning-SFT数据集进行修改，Complex_CoT列和Response列进行拼接，并加上文本结束标记：

In [108]:
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206180316919.png" alt="image-20250206180316919" style="zoom:50%;" />

在最小可行性实验中，我们可以只下载500条数据进行微调即可看出效果：

In [109]:
# dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)

dataset = load_dataset("./dataset/FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)

In [110]:
dataset[0]

{'Question': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?',
 'Complex_CoT': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine when she isn't under physical stress. Hmm, that's a clue that we're dealing with something related to pressure rather than a bladder muscle problem. \n\nThe fact that she underwent a Q-tip test is intriguing too. This 

然后进行结构化处理：

In [111]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

将数据集整理为如下形式：

In [112]:
dataset["text"][0]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her ab

- 数据集保存地址

默认情况下数据集保存在主目录下.cache文件夹中，数据文件格式如下所示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206195216257.png" alt="image-20250206195216257" style="zoom:50%;" />

#### 2.开启微调

&emsp;&emsp;然后即可把模型设置为微调模式：

In [113]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [114]:
# 增加的
FastLanguageModel.for_training(model) 

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [115]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

然后导入相关的库：

In [116]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

创建有监督微调对象：

In [117]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Converting train dataset to ChatML (num_proc=2): 100%|██████████| 500/500 [00:00<00:00, 2169.69 examples/s]
Applying chat template to train dataset (num_proc=2): 100%|██████████| 500/500 [00:01<00:00, 374.73 examples/s]
Tokenizing train dataset (num_proc=2): 100%|██████████| 500/500 [00:00<00:00, 1393.32 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


这段代码主要是用 **`SFTTrainer`** 进行 **监督微调（Supervised Fine-Tuning, SFT）**，适用于 `transformers` 和 `Unsloth` 生态中的模型微调：
**1. 导入相关库**
- **`SFTTrainer`**（来自 `trl` 库）：  
  - `trl`（Transformer Reinforcement Learning）是 Hugging Face 旗下的 `trl` 库，提供 **监督微调（SFT）** 和 **强化学习（RLHF）** 相关的功能。
  - `SFTTrainer` 主要用于 **有监督微调（Supervised Fine-Tuning）**，适用于 `LoRA` 等低秩适配微调方式。

- **`TrainingArguments`**（来自 `transformers` 库）：  
  - 这个类用于定义 **训练超参数**，比如批量大小、学习率、优化器、训练步数等。

- **`is_bfloat16_supported()`**（来自 `unsloth`）：  
  - 这个函数检查 **当前 GPU 是否支持 `bfloat16`（BF16）**，如果支持，则返回 `True`，否则返回 `False`。
  - `bfloat16` 是一种更高效的数值格式，在 **新款 NVIDIA A100/H100** 等 GPU 上表现更优。

**2. 初始化 `SFTTrainer` 进行模型微调**

##### **参数解析**
##### **① `SFTTrainer` 部分**
| 参数 | 作用 |
|------|------|
| `model=model` | 指定需要进行微调的 **预训练模型** |
| `tokenizer=tokenizer` | 指定 **分词器**，用于处理文本数据 |
| `train_dataset=dataset` | 传入 **训练数据集** |
| `dataset_text_field="text"` | 指定数据集中哪一列包含 **训练文本**（在 `formatting_prompts_func` 里处理） |
| `max_seq_length=max_seq_length` | **最大序列长度**，控制输入文本的最大 Token 数量 |
| `dataset_num_proc=2` | **数据加载的并行进程数**，提高数据预处理效率 |

##### **② `TrainingArguments` 部分**
| 参数 | 作用 |
|------|------|
| `per_device_train_batch_size=2` | 每个 **GPU/设备** 的训练批量大小（较小值适合大模型） |
| `gradient_accumulation_steps=4` | **梯度累积步数**（相当于 `batch_size=2 × 4 = 8`） |
| `warmup_steps=5` | **预热步数**（初始阶段学习率较低，然后逐步升高） |
| `max_steps=60` | **最大训练步数**（控制训练的总步数，此处总共约消耗60*8=480条数据） |
| `learning_rate=2e-4` | **学习率**（`2e-4` = 0.0002，控制权重更新幅度） |
| `fp16=not is_bfloat16_supported()` | 如果 **GPU 不支持 `bfloat16`，则使用 `fp16`（16位浮点数）** |
| `bf16=is_bfloat16_supported()` | 如果 **GPU 支持 `bfloat16`，则启用 `bfloat16`（训练更稳定）** |
| `logging_steps=10` | **每 10 步记录一次训练日志** |
| `optim="adamw_8bit"` | **使用 `adamw_8bit`（8-bit AdamW优化器）减少显存占用** |
| `weight_decay=0.01` | **权重衰减（L2 正则化）**，防止过拟合 |
| `lr_scheduler_type="linear"` | **学习率调度策略**（线性衰减） |
| `seed=3407` | **随机种子**（保证实验结果可复现） |
| `output_dir="outputs"` | **训练结果的输出目录** |

然后设置wandb（可选）：

In [120]:
! pip install wandb
import wandb

Looking in indexes: https://pypi.internal-mirrors.ucloud.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 33.4 MB/s eta 0:00:0000:0100:01
  Using cached https://pypi.internal-mirrors.ucloud.cn/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 69.5 MB/s eta 0:00:00
  Using cached https://pypi.internal-mirrors.ucloud.cn/packages/f4/3c/8cc1cc84deffa6e25d2d0c688ebb80635dfdbf1dbea3e30c541c8cf4d860/pydantic-2.10.6-py3-none-any.whl (431 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 28.1 MB/s eta 0:00:00
  Using cached https://pypi.internal-mirrors.ucloud.cn/packages/78/b6/6307fbef88d9b5ee7421e68d78a9f162e0da4900bc5f5793f6d3d0e34fb8/annotated_types-0.7.0-py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.1 MB/s

In [121]:
wandb.login(key="1a8d27529016174442b34dd0ae898bfa88d2156c")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc
wandb: Currently logged in as: www6v (www6v-vip) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

然后开始微调：

In [122]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.941600
20,1.471300
30,1.410600
40,1.320400
50,1.355500
60,1.322600


此时wandb中显示内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200441907.png" alt="image-20250206200441907" style="zoom:50%;" />

In [123]:
trainer_stats

TrainOutput(global_step=60, training_loss=1.4703421751658121, metrics={'train_runtime': 157.2751, 'train_samples_per_second': 3.052, 'train_steps_per_second': 0.381, 'total_flos': 1.8014312853602304e+16, 'train_loss': 1.4703421751658121})

注意，unsloth在微调结束后，会自动更新模型权重（在缓存中），因此无需手动合并模型权重即可直接调用微调后的模型：

In [124]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [125]:
inputs = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [126]:
print(response[0].split("### Response:")[1])


<think>
Okay, let's see. We have a 61-year-old woman with a history of involuntary urine loss, especially during things like coughing or sneezing. That sounds like a classic presentation of urge incontinence. Now, when we think about what's happening here, it's likely that her bladder is contracting involuntarily. This could be due to an overactive bladder, which is often caused by an underlying condition like a neurological issue or a problem with the pelvic floor muscles.

Now, let's talk about the Q-tip test. This test is designed to check how much urine is left in the bladder after it's emptied. If it's positive, it means there's residual urine, which can contribute to incontinence because it increases the chances of accidental leakage. So, if the Q-tip test is positive, it's pretty clear that her bladder isn't emptying completely.

Then there's the cystometry part. This test is used to measure how much urine remains in the bladder after it's emptied. If there's residual volume, i

测试第二个问题问答效果：

In [127]:
inputs = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [128]:
print(response[0].split("### Response:")[1])


<think>
Okay, so we've got this patient here experiencing sudden chest pain that's spreading to their neck and left arm. That's a classic symptom of a heart attack. And they've got a history of high cholesterol and heart disease, which is a red flag for someone at risk for a cardiac event. Plus, their troponin levels are elevated, which directly points to cardiac damage, and they're tachycardic, meaning their heart is racing. These all line up with what we'd expect from a coronary artery blockage.

Now, we need to figure out which artery is most likely the culprit here. Coronary artery disease usually affects the left main artery, left anterior descending artery, or the right coronary artery. Given that the pain is radiating to the neck and left arm, it's telling us that the left main artery is probably the one involved. This artery is connected to several branches that supply the left side of the heart, which explains the widespread pain.

So, putting it all together, the most likely

> 此时模型认为“左主冠状动脉最可能是该患者症状的罪魁祸首”，但实际上应该是“左前降支（LAD）动脉受累”导致。

能够发现，第一个问题回答更加规范，并且回答正确。但第二个问题仍然回答错误。由此可以考虑继续进行大规模微调。不过在此之前，我们可以将现在小规模微调的模型进行本地保存。

#### 3.模型合并

此时本地保存的模型权重在`outputs`文件夹中：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206195427494.png" alt="image-20250206195427494" style="zoom:50%;" />

然后可使用如下代码进行模型权重合并：

In [129]:
new_model_local = "DeepSeek-R1-Medical-COT-Tiny"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 70.71 out of 125.72 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 67.87it/s]


Unsloth: Saving tokenizer... Done.
Done.


保存结束后，即可在当前文件夹中看到对应模型：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200056475.png" alt="image-20250206200056475" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200157110.png" alt="image-20250206200157110" style="zoom:50%;" />

然后即可将其推送到huggingface上并保存为GGUF格式文件并进行调用。

In [130]:
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q4_k_m")
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q8_0")
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "f16")

### 三、完整高效微调实验

&emsp;&emsp;接下来我们尝试带入全部数据进行高效微调，以提升模型微调效果。

In [131]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [132]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

此时读取全部数据

In [133]:
dataset = load_dataset("./dataset/FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

Map: 100%|██████████| 25371/25371 [00:00<00:00, 37925.20 examples/s]


"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her ab

In [134]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


这里设置epoch为3，遍历3次数据集：

In [135]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs = 1,   # 3 
        warmup_steps=5,
        # max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Converting train dataset to ChatML (num_proc=2): 100%|██████████| 25371/25371 [00:00<00:00, 32660.41 examples/s]
Applying chat template to train dataset (num_proc=2): 100%|██████████| 25371/25371 [00:01<00:00, 13687.14 examples/s]
Tokenizing train dataset (num_proc=2): 100%|██████████| 25371/25371 [00:09<00:00, 2714.73 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [136]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,371 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 3,171
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.333900
20,1.317900
30,1.287300
40,1.313600
50,1.330800
60,1.296900
70,1.292200
80,1.247300
90,1.291200
100,1.242200


这里总共训练约2个小时。

In [137]:
trainer_stats

TrainOutput(global_step=3171, training_loss=1.205654080641153, metrics={'train_runtime': 7429.134, 'train_samples_per_second': 3.415, 'train_steps_per_second': 0.427, 'total_flos': 9.317611621994988e+17, 'train_loss': 1.205654080641153})

带入两个问题进行测试，均有较好的回答效果：

In [138]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Alright, let's think about this. We have a 61-year-old woman who's been dealing with involuntary urine leakage whenever she coughs or sneezes. That sounds like stress incontinence. You know, when there's pressure on the bladder, like from coughing, it makes sense she'd leak.

Now, stress incontinence is all about the urethra, which is the narrow tube that carries urine from the bladder. The urethra doesn't quite close up tightly enough, so when there's pressure, like during a cough, it just lets a bit of urine slip out.

Okay, so what about the Q-tip test? This test is used to check how well the urethra closes. If it doesn't close properly, that would explain the stress incontinence. It's like the urethra isn't strong enough to hold up against pressure.

Thinking about what cystometry might show, it's used to see how the bladder behaves. We're interested in two things: residual volume, which is the amount of urine left in the bladder after it empties, and detrusor contractions

In [139]:
question = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, let's think about this. We've got a patient with sudden chest pain that's shooting down to the neck and arm. That's pretty classic for a heart attack, right? The pain is usually a big clue here. And this patient also has a history of high cholesterol and coronary artery disease. That's like putting a big red flag up for potential heart issues.

Now, let's see what else we've got. The elevated troponin levels are another huge red flag. Troponin is like the body's signal that something's really wrong with the heart muscle. And then there's the tachycardia, which means the heart is beating really fast. All these things together scream acute coronary syndrome, especially a myocardial infarction.

So, what's happening here? It's probably that a blockage in one of the coronary arteries is causing all this trouble. Now, which artery could it be? Hmm, there's a pattern here. The pain radiates to the left arm and neck, which is usually a hint that we're dealing with the left ante

最后进行模型权重保存：

In [140]:
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 69.46 out of 125.72 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 69.96it/s]


Unsloth: Saving tokenizer... Done.
Done.


<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206201842044.png" alt="image-20250206201842044" style="zoom:50%;" />